# 🤖✨ Chatbot Assistant with Persistent Memory

**Local, tool-enabled conversational AI using LangChain v1.0+, Ollama, and Gradio v6.1.0**

---

## Overview

Fully local, production-ready chatbot with persistent memory and tool-calling capabilities. Built for open-source LLMs, privacy-sensitive environments, and offline deployments.

---

## Architecture

### 🧠 Memory Management
- **Checkpointer**: `MemorySaver` from LangGraph stores conversation state
- **Thread-based**: Isolated `thread_id` per session for conversation history
- **Automatic**: Memory injected/saved on each agent invocation

### 🔧 Agent & Tools
- **Agent**: `create_agent()` from LangChain v1.0+ with native tool support
- **Tools**: Wikipedia, Tavily search, Weather, and custom tools
- **System Prompt**: Configurable personality

### 🤖 Local LLM (Ollama)
- **Provider**: `ChatOllama` connects to local Ollama server
- **Model**: Configurable (default: `qwen2.5:3b`)
- **Temperature**: `0.0` for deterministic responses

### 🎨 UI (Gradio)
- **Layout**: `gr.Blocks` for flexible components
- **State**: Session-based with `gr.State`
- **Features**: Chat history, message counter, clear button, examples

---

## Key Features

✅ **Persistent Memory** - Full conversation context maintained  
✅ **Tool Calling** - Wikipedia, Tavily web search, custom tools  
✅ **Local Execution** - No mandatory API keys needed or cloud dependencies  
✅ **Session Isolation** - Independent memory per session  
✅ **Error Handling** - Graceful failures with logging  
✅ **Modular Design** - Clean separation of concerns

---

## How It Works

```python
# Initialize with checkpointer for memory
agent = create_agent(
    model=ChatOllama(...),
    tools=[wikipedia_tool, tavily_search, ...],
    system_prompt="You are helpful...",
    checkpointer=MemorySaver()
)

# Invoke with thread_id for conversation history
result = agent.invoke(
    {"messages": [HumanMessage(content=user_input)]},
    config={"configurable": {"thread_id": session_id}}
)
```

The `MemorySaver` checkpointer loads previous messages, saves new messages, and provides full context to the agent.

---

## Installation

```bash
pip install langchain langchain-ollama langchain-core langgraph gradio langchain-community

# Start Ollama and pull model
ollama serve
ollama pull qwen2.5:3b
```

---

## Usage

```python
from langchain_community.tools import WikipediaQueryRun, TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper

# Define tools
tools = [
    WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
    TavilySearchResults(max_results=3)
]

# Create and launch chatbot
bot = chatbot_demo(tools=tools, model_name="qwen2.5:3b")
demo = bot.create_ui()
demo.launch()
```

---

## Design Decisions

| Aspect | Choice | Rationale |
|--------|--------|-----------|
| **Memory** | `MemorySaver` checkpointer | Modern LangChain v1.0+ pattern |
| **Agent** | `create_agent()` | Replaces deprecated `create_react_agent` |
| **Config Key** | `thread_id` | LangGraph standard for threads |
| **LLM Provider** | Ollama | Privacy, offline, cost-free |
| **UI Framework** | Gradio | Rapid prototyping, built-in chat |

---

## Extending the System

### Add Custom Tools
```python
from langchain.tools import tool

@tool
def calculator(expression: str) -> str:
    """Evaluate mathematical expressions"""
    return str(eval(expression))

tools = [calculator, tavily_search, ...]
```

### Change Model
```python
bot = chatbot_demo(tools=tools, model_name="llama2:13b")
```

---

## Use Cases

- 🏠 **Privacy-first applications** - All data stays local
- 🔍 **Research assistants** - Wikipedia + Tavily web search
- 🧪 **Experimentation** - Flexible tool integration
- 🚀 **Production prototypes** - Scalable foundation

---

## Technical Notes

- **Message Replay**: Checkpointer handles history automatically
- **State Separation**: UI state (Gradio) ≠ Agent state (LangGraph)
- **Thread Safety**: Each `thread_id` is isolated for concurrent sessions
- **Tool Integration**: Tavily enables real-time web search capabilities

---

## Summary

Modern LangChain implementation demonstrating:
- ✨ Clean agent architecture with `create_agent()`
- 💾 Persistent memory via `MemorySaver`
- 🔧 Tool-augmented reasoning (Wikipedia, Tavily search)
- 🎯 Production-ready error handling

Perfect foundation for local, tool-enabled chatbots with memory.

*Based on the final project from the course **“Functions, Tools and Agents with LangChain”** by deeplearning(dot)ai* fully updated to LangChain v1+, with a new custom UI based on Gradio and a local LLM running via Ollama.

In [ ]:
import gradio as gr
import param
import uuid

from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
class chatbot_demo(param.Parameterized):
    """
    Chatbot Assistant with Persistent Conversational Memory.

    Memory is implemented using LangGraph's MemorySaver checkpointer.
    Each UI session gets an isolated thread for conversation history.
    """

    def __init__(self, tools, model_name="qwen2.5:3b", **params):
        super(chatbot_demo, self).__init__(**params)
        self.tools = tools

        # ------------------------------------------------------------
        # CHECKPOINTER FOR MEMORY
        # ------------------------------------------------------------
        # MemorySaver stores conversation state in memory
        self.checkpointer = MemorySaver()

        # Track session IDs (thread IDs in LangGraph terminology)
        self._session_ids = set()

        # ------------------------------------------------------------
        # SYSTEM PROMPT
        # ------------------------------------------------------------
        self.system_prompt = (
            "You are a helpful and friendly assistant with access to:\n"
            "- Weather Search (current conditions & forecasts)\n"
            "- Wikipedia (factual information)\n"
            "- Tavily (web search)(if available)\n"
            "\n"
            "When tools return source URLs, cite them as markdown links: [Source Name](URL)\n"
            "ONLY include links that are actually provided by the tools - never create or guess URLs.\n"
            "Use conversation context to provide relevant, personalized responses."
        )

        # ------------------------------------------------------------
        # MODEL
        # ------------------------------------------------------------
        self.model = ChatOllama(
            model=model_name,
            temperature=0,  # increase for less deterministic results
            base_url="http://localhost:11434"
        )

        # ------------------------------------------------------------
        # CREATE AGENT WITH MEMORY (via checkpointer)
        # ------------------------------------------------------------
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            system_prompt=self.system_prompt,
            checkpointer=self.checkpointer  # This enables memory!
        )

    def process_chain(self, message, session_id):
        """
        Process user message and return response with session-wise memory.

        Args:
            message: Current user message
            session_id: Unique session identifier (maps to thread_id)

        Returns:
            Bot response string
        """
        if not message:
            return ""

        try:
            # Track this session
            self._session_ids.add(session_id)

            # Invoke agent with thread_id for memory
            # The checkpointer automatically handles conversation history
            result = self.agent.invoke(
                {
                    "messages": [HumanMessage(content=message)]
                },
                config={
                    "configurable": {
                        "thread_id": session_id  # thread_id is the key for memory
                    }
                }
            )

            # Extract bot response (last AI message)
            for msg in reversed(result["messages"]):
                if isinstance(msg, AIMessage):
                    return msg.content
            
            # Fallback
            return result["messages"][-1].content

        except Exception as e:
            import traceback
            error_details = traceback.format_exc()
            print(f"Error details:\n{error_details}")
            return f"Sorry, I encountered an error: {str(e)}"

    def clear_history(self, session_id):
        """Clear conversation memory for a session"""
        if session_id:
            # To clear memory, we need to get the checkpoint and clear it
            # For MemorySaver, we can't directly clear, but we can track cleared sessions
            # and create a new thread_id to effectively "clear" the conversation
            pass
        return None

    def get_history_summary(self, session_id):
        """Get a summary of conversation history for a session"""
        if not session_id or session_id not in self._session_ids:
            return "Messages in memory: 0"
        
        try:
            # Try to get state to count messages
            config = {"configurable": {"thread_id": session_id}}
            state = self.agent.get_state(config)
            if state and "messages" in state.values:
                msg_count = len(state.values["messages"])
                return f"Messages in memory: {msg_count}"
        except:
            pass
        
        return "Messages in memory: N/A"

    def create_ui(self):
        """Create and return Gradio ChatInterface"""

        with gr.Blocks() as demo:
            gr.Markdown("# 🤖✨ Chatbot Assistant with Persistent Memory")
            gr.Markdown("Ask me anything! I'll remember our conversation per session.")

            chatbot = gr.Chatbot(height=500)

            msg = gr.Textbox(
                placeholder="Type your message here...",
                label="Message",
                lines=2
            )

            # Session state (one per browser session)
            session_state = gr.State(value=None)

            with gr.Row():
                submit = gr.Button("Send", variant="primary")
                clear = gr.Button("Clear History")

            # History info display
            history_info = gr.Markdown("Messages in memory: 0")

            gr.Examples(
                examples=[
                    "Latest trends in autonomous AI agents and tool use",
                    "Warsaw weather (52.2297, 21.0122) - should I bring a jacket?",
                    "Recent breakthroughs in AI reasoning and planning",
                    "Search Wikipedia for startup accelerator programs",
                    "Weather in SF (37.7749, -122.4194) for client meeting",
                    "Find recent news about LangChain developments",
                    "What makes an AI system 'agentic'?",
                    "Recap our earlier conversation about startups",
                    "Explain tool-calling in modern AI systems",
                    "Explain the architecture of retrieval-augmented generation",
                    "What's the current state of multimodal AI models?",
                    "Weather in Paris, France for tomorrow's conference?",
                    "Find current news about breakthrough in quantum computing",
                    "Create a summary of all topics we've discussed today",
                    "Search web for today's major tech announcements",
                    "Recent advances in reinforcement learning from human feedback"
                ],
                inputs=msg
            )

            def respond(message, chat_history, session_id):
                if not message:
                    return "", chat_history, session_id, history_info.value

                # Create session id once per browser session
                if session_id is None:
                    session_id = str(uuid.uuid4())

                if chat_history is None:
                    chat_history = []

                bot_message = self.process_chain(message, session_id)

                # Update UI chat history
                chat_history = chat_history + [
                    {"role": "user", "content": message},
                    {"role": "assistant", "content": bot_message}
                ]

                return (
                    "",
                    chat_history,
                    session_id,
                    self.get_history_summary(session_id)
                )

            def clear_all(chat_history, session_id):
                # To truly clear history with MemorySaver, create a new session
                new_session_id = str(uuid.uuid4()) if session_id else None
                return [], new_session_id, "Messages in memory: 0"

            # Event handlers
            submit.click(
                respond,
                [msg, chatbot, session_state],
                [msg, chatbot, session_state, history_info]
            )
            msg.submit(
                respond,
                [msg, chatbot, session_state],
                [msg, chatbot, session_state, history_info]
            )
            clear.click(
                clear_all,
                [chatbot, session_state],
                [chatbot, session_state, history_info]
            )

        return demo

### Tools available to the chatbot

In [ ]:
from langchain.tools import tool

# Create your custom tool
@tool
def create_your_tool(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
# Import existing tools
from tools.web_retrievers import search_wikipedia, retrieve_tavily
from tools.meteo import get_weather_conditions

tools = [retrieve_tavily, search_wikipedia, get_weather_conditions]

### User Interface (Gradio)

In [ ]:
import gradio as gr

# Run the Chatbot UI 
if __name__ == "__main__":
    
    # Create agent with tools
    chatbot = chatbot_demo(tools=tools, model_name="qwen2.5:3b")
    
    # Launch interface
    interface = chatbot.create_ui()
    interface.launch(
        share=False, 
        debug=True,
        theme=gr.themes.Citrus()
    )